Universidad del Valle de Guatemala <br>
Security Data Science

# LABORATORIO 3
## Clasificación de Malware

Walter Saldaña #19897
___

importar librerias

In [50]:
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score 
from sklearn.metrics import accuracy_score, precision_score, recall_score
from xgboost import XGBClassifier

cargar datos

In [2]:
df = pd.read_csv('VirusSample.csv')

In [3]:
df.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus


# Análisis exploratorio

In [4]:
df.describe()

,file,api,class
count,9795,9795,9795
unique,9795,2980,13
top,7ff49f2f0912352416b05c010f35f402cc79feed,"LoadLibraryA,GetProcAddress",Trojan
freq,1,1007,6153


In [5]:
df.dtypes

file     object
api      object
class    object
dtype: object

In [6]:
df['class'].unique()

array(['Virus', 'Riskware', 'Spyware', 'Downloader', 'Dropper', 'Agent',
       'Adware', 'Trojan', 'Backdoor', 'Worms', 'Ransomware', 'Keylogger',
       'Crypt'], dtype=object)

In [7]:
df.groupby('class')['file'].count().sort_values()

class
Keylogger        1
Crypt            2
Dropper          4
Riskware         4
Ransomware      10
Spyware         11
Downloader      31
Agent          102
Adware         222
Worms          441
Backdoor       447
Virus         2367
Trojan        6153
Name: file, dtype: int64

In [8]:
df.isnull().sum()

file     0
api      0
class    0
dtype: int64

In [9]:
df.isna().sum()

file     0
api      0
class    0
dtype: int64

# Preprocesamiento

In [10]:
df['api'] = df['api'].str.split(',')
df.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"[IntersectRect, GetCurrentProcess, GetVersion]",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"[GetCaretBlinkTime, CountClipboardFormats, Get...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"[VarR8Pow, GetClipboardViewer, GetInputDesktop...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"[SetTraceCallback, CopyAcceleratorTableW, GetP...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"[SHLoadNonloadedIconOverlayIdentifiers, VarUI8...",Virus


In [11]:
le = preprocessing.LabelEncoder()
df['class'] = le.fit_transform(df['class'])
df.sample(10)

,file,api,class
3958,031547b2c2b281d89d7246fadfa891ed0a69969a,"[GetModuleHandleA, GetProcAddress, fabs, InitC...",10
9773,c0e3b32b6c0c985a21dd297c1cff3eab4a848d6396c639...,"[ZwQueryInformationFile, NtEnumerateValueKey, ...",11
4086,e9ef933e632c647b63aae53ed139ef074c679a0b,"[GetModuleHandleW, GetLocalTime, GetComputerNa...",10
468,3dd1847be1899d1cebddd54693411ddd1f359489,"[LoadLibraryA, GetProcAddress, VirtualProtect,...",0
8480,2d44c57f6a6429e5c447f507a8fe8a241db0eeaf,"[MethCallEngine, EVENT_SINK_AddRef, DllFunctio...",10
6111,d1bbc5524839e2f9f0dc9219f30af7b9e9813e6f,"[DeleteCriticalSection, EnterCriticalSection, ...",10
5535,1281a48aae4415824dbe52d51cf7864b20c0f235,"[LoadLibraryA, GetProcAddress, VirtualProtect,...",10
7218,178ce1e085df8d7ca05b4b530743ed397956aff3,"[DeleteCriticalSection, EnterCriticalSection, ...",10
6721,8c2d9a511da6e19049018673c8b772507fd67bb7,"[LoadLibraryA, GetProcAddress, VirtualProtect,...",10
2503,b13c6ea1198f728bdde78883e03308eba6eed0a1,"[OutputDebugStringA, ZwCompareTokens, GetClipb...",11


# Ingeniería de características

In [12]:
'''
dummies = pd.Series()
jump_value = 1000
max_value = df['api'].count()

for i in range(0, max_value, jump_value):
    nxt = i + jump_value
    limit = nxt if nxt < max_value else None
    dummies_ext = df.iloc[i:limit]['api'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()
    dummies = pd.concat([dummies, dummies_ext])
    print(f'Transformed {nxt} - {len(dummies_ext.index)}')
'''

"\ndummies = pd.Series()\njump_value = 1000\nmax_value = df['api'].count()\n\nfor i in range(0, max_value, jump_value):\n    nxt = i + jump_value\n    limit = nxt if nxt < max_value else None\n    dummies_ext = df.iloc[i:limit]['api'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()\n    dummies = pd.concat([dummies, dummies_ext])\n    print(f'Transformed {nxt} - {len(dummies_ext.index)}')\n"

In [13]:
# dummies = dummies.fillna(0)
# dummies

In [14]:
# dummies.to_csv('dummies.csv')

In [15]:
dummies = pd.read_csv('dummies.csv')

In [16]:
col_names = dummies.columns
col_names

Index(['Unnamed: 0', '0', '??0?$basic_ios@DU?$char_traits@D@std@@@std@@IAE@XZ',
       '??0?$basic_ios@DU?$char_traits@D@std@@@std@@IEAA@XZ',
       '??0?$basic_istream@DU?$char_traits@D@std@@@std@@QEAA@PEAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z',
       '??0?$basic_ostream@DU?$char_traits@D@std@@@std@@QAE@PAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z',
       '??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IAE@XZ',
       '??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IEAA@XZ',
       '??0_Container_base12@std@@QAE@XZ', '??0_Locinfo@std@@QAE@HPBD@Z',
       ...
       'rtcIsArray', 'rtcIsMissing', 'rtcIsObject', 'rtcJoin', 'rtcLeftVar',
       'rtcPackTime', 'rtcRandomNext', 'rtcVarBstrFromAnsi',
       'rtcVarDateFromVar', 'wcscspn'],
      dtype='object', length=7967)

In [17]:
df = pd.concat([df, dummies], axis=1)
del df['Unnamed: 0']
del df['0']
del df['api']
del df['file']
df.head()

,class,??0?$basic_ios@DU?$char_traits@D@std@@@std@@IAE@XZ,??0?$basic_ios@DU?$char_traits@D@std@@@std@@IEAA@XZ,??0?$basic_istream@DU?$char_traits@D@std@@@std@@QEAA@PEAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z,??0?$basic_ostream@DU?$char_traits@D@std@@@std@@QAE@PAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z,??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IAE@XZ,??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IEAA@XZ,??0_Container_base12@std@@QAE@XZ,??0_Locinfo@std@@QAE@HPBD@Z,??0_Lockit@std@@QAE@H@Z,...,rtcIsArray,rtcIsMissing,rtcIsObject,rtcJoin,rtcLeftVar,rtcPackTime,rtcRandomNext,rtcVarBstrFromAnsi,rtcVarDateFromVar,wcscspn
0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Modelo 1: Support Vector Machine

## Implementación y validación

Split data

In [18]:
X = df.loc[:, df.columns != 'class']
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

Crear y entrenar el modelo

In [19]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

Validar predicciones

In [20]:
y_pred = clf.predict(X_test)

## Validación cruzada

In [21]:
# k_folds = KFold(n_splits = 10)
scores = cross_val_score(clf, X, y, cv = 10)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

/home/walt/.local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Cross Validation Scores:  [0.94693878 0.95918367 0.93877551 0.93469388 0.93265306 0.93054137
 0.93564862 0.93769152 0.93667007 0.94484168]
Average CV Score:  0.9397638156386151
Number of CV Scores used in Average:  10


## Métricas

In [23]:
accuracy_score(y_test, y_pred)

0.9445389588295339

In [30]:
precision_score(y_test, y_pred, average='weighted', zero_division=False)

0.9405462285134326

In [31]:
recall_score(y_test, y_pred, average='weighted', zero_division=False)

0.9445389588295339

# Modelo 2: Extreme Gradient Booster

## Implementación y validación

In [82]:
bst = XGBClassifier(n_estimators=10, max_depth=2, learning_rate=1, objective='hist')

In [83]:
bst.fit(X_train, y_train)

/home/walt/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/walt/.local/lib/python3.9/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[20:50:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [84]:
y_pred_xgb = bst.predict(X_test)

## Validación cruzada

## Métricas

In [85]:
accuracy_score(y_test, y_pred_xgb)

0.9118747873426335

In [86]:
precision_score(y_test, y_pred_xgb, average='weighted', zero_division=False)

0.8970088120384435

In [87]:
recall_score(y_test, y_pred_xgb, average='weighted', zero_division=False)

0.9118747873426335

___
# Conclusiones
**El artículo “New Datasets for Dinamyc Malware Classification” sirve como un benchmark para comparar modelos de clasificación, ¿se lograron obtener mejores métricas que las obtenidas en el artículo para la clasificación de malware?**